In [1]:
import torch
from torch import nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 1000),
            nn.ReLU(),
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 1000),
            nn.ReLU(),
            nn.Linear(1000, 28*28),
            nn.Sigmoid()
        )

    def forward(self, flat):
        return self.model(flat)

In [2]:
model = Autoencoder()
model.load_state_dict(torch.load("models/ae-1.pth", map_location="cpu"))

<All keys matched successfully>

In [7]:
def matrix_operator_norm1(A):
    return torch.linalg.matrix_norm(A, 1)
def cond1(A):
    A_inverse = torch.linalg.pinv(A)
    return matrix_operator_norm1(A)*matrix_operator_norm1(A_inverse)

@torch.no_grad()
def combined_cond(model):
    weights = [param for name, param in model.named_parameters() if "weight" in name]
    cond_nums = [cond1(param).item() for param in weights]
    return max(cond_nums)
combined_cond(model)

54168.84375